## Experiments: Comparing RSAST with other methods

In [319]:
import sys 
import os 
#add sast library path
file_path = os.path.dirname(os.getcwd())+"/sast"

#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.path.dirname(os.getcwd())+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\sast"


#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

#add cd_diagram library path
file_path = os.path.dirname(os.getcwd())+"\cd_diagram"

#file_path = r"C:\Users\Public\random_sast\cd_diagram"
sys.path.append(file_path)


file_path = os.path.dirname(os.getcwd())+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"\cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

file_path = os.getcwd()+"/cd_diagram"
#file_path = r"C:\Users\Public\random_sast\sast"
sys.path.append(file_path)

sys.path


['/home/nirojasvar/random_sast/ExperimentationRSAST',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python310.zip',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/lib-dynload',
 '',
 '/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nirojasvar/random_sast/cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\cd_diagram',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/cd_diagram',
 '/home/nirojasvar/random_sast/sast',
 '/home/nirojasvar/random_sast\\sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST/sast',
 '/home/nirojasvar/random_sast/ExperimentationRSAST\\sast',
 '/home/nirojasvar/random_sast\\cd_diagram',
 '/home/nir

In [320]:
from sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd
from sktime.transformations.panel.shapelet_transform import ShapeletTransform
import matplotlib.pyplot as plt
import os
import seaborn as sns
import math
hyperparameter='ACF&PACF: n_random_points=10 nb_inst_per_class=10'

### Reading Datasets

In [321]:

# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsByClassifier'

# Create an empty list to store the dataframes
dfs = []

header=['dataset']
res = [str(ele) for ele in range(30)]
header.extend(res)

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath, names=header,index_col=False)

        # Append the dataframe to the list
        df['filename']=filename
        dfs.append(df)


# Concatenate all the dataframes in the list into one dataframe
df_other_methods = pd.concat(dfs, ignore_index=True)

res=res[:10]

df_other_methods["avg"]=df_other_methods[res].mean(axis=1)

df_other_methods=df_other_methods[['filename','dataset','avg']]
df_other_methods['method']=df_other_methods['filename'].str.split('.').str[0]+"_UCR_RES_10"
df_other_methods
df_other_methods=df_other_methods.rename(columns={'avg':'score'})
df_other_methods=df_other_methods[['dataset','score','method']]

df_other_methods

/tmp/ipykernel_2911/2425501165.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
/tmp/ipykernel_2911/2425501165.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
/tmp/ipykernel_2911/2425501165.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
/tmp/ipykernel_2911/2425501165.py:17: ParserWarning: Length of header or names does not match length of data. This leads to a loss of data with index_col=False.
  df = pd.read_csv(filepath, names=header,index_col=False)
/tmp/ipykernel_2911/2425501165.py:17: ParserWarning: Length of header or names does not match length of data. This leads

,dataset,score,method
0,Adiac,0.809207,HIVE-COTE_UCR_RES_10
1,ArrowHead,0.897143,HIVE-COTE_UCR_RES_10
2,Beef,0.763333,HIVE-COTE_UCR_RES_10
3,BeetleFly,0.965000,HIVE-COTE_UCR_RES_10
4,BirdChicken,0.930000,HIVE-COTE_UCR_RES_10
...,...,...,...
1778,Wine,0.888889,RotF_UCR_RES_10
1779,WordSynonyms,0.587774,RotF_UCR_RES_10
1780,Worms,0.606494,RotF_UCR_RES_10
1781,WormsTwoClass,0.657143,RotF_UCR_RES_10


In [322]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ResultsSummarized/UCR_ARCHIVE_singleTrainTest_ALL_METHODS_MOD.csv'

df = pd.read_csv(filepath)
df = df.melt(id_vars=["Unnamed: 0"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Unnamed: 0':'dataset'})
df['method'] = df['method']+"_UCR_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])
df[df.dataset=="StarlightCurves"]

,dataset,score,method


In [323]:
# Set directory where the csv files are located
"""
filepath = os.getcwd()+'/ResultsSummarized/UCR_ARCHIVE_100RESAMPLES_ALL_METHODS_MOD.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Unnamed: 0"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Unnamed: 0':'dataset'})
df['method'] = df['method']+"_UCR_AVG_100RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])
"""

'\nfilepath = os.getcwd()+\'/ResultsSummarized/UCR_ARCHIVE_100RESAMPLES_ALL_METHODS_MOD.csv\'\ndf = pd.read_csv(filepath)\ndf = df.melt(id_vars=["Unnamed: 0"], \n        var_name="method", \n        value_name="score")\ndf = df.rename(columns={\'Unnamed: 0\':\'dataset\'})\ndf[\'method\'] = df[\'method\']+"_UCR_AVG_100RES"\ndf=df[[\'dataset\',\'score\',\'method\']]\ndf_other_methods=pd.concat([df_other_methods,df])\n'

In [324]:

# Set directory where the csv files are located
#df_other_methods=df_other_methods[df_other_methods.method!="STC"]
filepath = os.getcwd()+'/ResultsSummarized/SAST_PAPER_singleTrainTest_SAST_ST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'folds:':'dataset','STC':'score'})
df['method']="STC_PAPERSAST_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])



In [325]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ResultsSummarized/SAST_PAPER_singleTrainTest_SAST_ST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'folds:':'dataset','0':'score'})
df['method']="SAST_PAPERSAST_TRAIN_TEST"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])

In [326]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ResultsSummarized/ROCKET_PAPER_10RESAMPLES_BAKE_OFF.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Dataset"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Dataset':'dataset'})
df['method'] = df['method']+"_PAPERROCKET_AVG_10RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [327]:
# Set directory where the csv files are located


filepath = os.getcwd()+'/ResultsSummarized/ROCKET_PAPER_10RESAMPLES_2018_DS.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["Dataset"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'Dataset':'dataset'})
df['method'] = df['method']+"_PAPERROCKET_AVG_10RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [328]:
# Set directory where the csv files are located


filepath = os.getcwd()+'/ResultsSummarized/RDST_PAPER_30RESAMPLES_ALL_METHODS.csv'
df = pd.read_csv(filepath)
df = df.melt(id_vars=["dataset_name"], 
        var_name="method", 
        value_name="score")
df = df.rename(columns={'dataset_name':'dataset'})
df['method'] = df['method']+"_PAPERRDST_AVG_30RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [329]:
# Set directory where the csv files are located

filepath = os.getcwd()+'/ResultsSummarized/RDST_PAPER_30RESAMPLES_RDST.csv'
df = pd.read_csv(filepath)
df = df.rename(columns={'dataset':'dataset','acc_mean':'score','model':'method'})
df['method'] = df['method']+"_PAPERRDST_AVG_30RES"
df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])


In [330]:

# Set directory where the csv files are located
directory = os.getcwd()+'/ResultsSummarized/Server17_Comparison_Other_Methods_singleTrainTest'

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file starts with "df_overall_tunning" and ends with ".csv"
    if filename.endswith(".csv"):
        # Read the csv file into a dataframe
        filepath = os.path.join(directory, filename)
        df = pd.read_csv(filepath)

        # Append the dataframe to the list
        
        dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df = pd.concat(dfs, ignore_index=True)
df=df[['dataset_name','classifier_name','accuracy']]

df=pd.pivot_table(df, index=['dataset_name','classifier_name'],aggfunc='mean').reset_index()
df['method']=df['classifier_name'].str.split('_').str[0]+"_RSAST_TRAIN_TEST"
df=df.rename(columns={'dataset_name':'dataset','accuracy':'score'})

df=df[['dataset','score','method']]
df_other_methods=pd.concat([df_other_methods,df])
df

,dataset,score,method
0,ACSF1,0.918000,RDST_RSAST_TRAIN_TEST
1,ACSF1,0.890000,Rocket_RSAST_TRAIN_TEST
2,Adiac,0.727877,RDST_RSAST_TRAIN_TEST
3,Adiac,0.784143,Rocket_RSAST_TRAIN_TEST
4,AllGestureWiimoteX,0.768857,RDST_RSAST_TRAIN_TEST
...,...,...,...
251,Worms,0.729870,Rocket_RSAST_TRAIN_TEST
252,WormsTwoClass,0.763636,RDST_RSAST_TRAIN_TEST
253,WormsTwoClass,0.800000,Rocket_RSAST_TRAIN_TEST
254,Yoga,0.928333,RDST_RSAST_TRAIN_TEST


In [331]:
df_other_methods['method'].unique()

array(['HIVE-COTE_UCR_RES_10', 'BOSS_UCR_RES_10', 'Flat-COTE_UCR_RES_10',
       'DTD_C_UCR_RES_10', 'ST_UCR_RES_10', 'TSF_UCR_RES_10',
       'EE_UCR_RES_10', 'LS_UCR_RES_10', 'SAXVSM_UCR_RES_10',
       'BoP_UCR_RES_10', 'MSM_UCR_RES_10', 'DD_DTW_UCR_RES_10',
       'WDTW_UCR_RES_10', 'CID_DTW_UCR_RES_10', 'DTW_F_UCR_RES_10',
       'FS_UCR_RES_10', 'LPS_UCR_RES_10', 'TWE_UCR_RES_10',
       'DDTW_UCR_RES_10', 'TSBF_UCR_RES_10', 'RotF_UCR_RES_10',
       'NB_UCR_TRAIN_TEST', 'C45_UCR_TRAIN_TEST', 'SVML_UCR_TRAIN_TEST',
       'SVMQ_UCR_TRAIN_TEST', 'BN_UCR_TRAIN_TEST', 'RandF_UCR_TRAIN_TEST',
       'RotF_UCR_TRAIN_TEST', 'MLP_UCR_TRAIN_TEST',
       'Euclidean_1NN_UCR_TRAIN_TEST', 'DTW_R1_1NN_UCR_TRAIN_TEST',
       'DTW_Rn_1NN_UCR_TRAIN_TEST', 'DDTW_R1_1NN_UCR_TRAIN_TEST',
       'DDTW_Rn_1NN_UCR_TRAIN_TEST', 'ERP_1NN_UCR_TRAIN_TEST',
       'LCSS_1NN_UCR_TRAIN_TEST', 'MSM_1NN_UCR_TRAIN_TEST',
       'TWE_1NN_UCR_TRAIN_TEST', 'WDDTW_1NN_UCR_TRAIN_TEST',
       'WDTW_1NN_UCR_TRAIN_T

In [332]:

#Comparison RSAST, SAST and STC (No resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['ST_UCR_TRAIN_TEST','SAST_PAPERSAST_TRAIN_TEST','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and Shapelet Methods (10 resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['R_DST_PAPERRDST_AVG_30RES','FS_UCR_RES_10', 'LS_UCR_RES_10','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and Other Categories (distance, interval, dictionary)(10 resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['EE_UCR_RES_10', 'TSBF_UCR_RES_10','BOSS_UCR_RES_10','DTW_R1_1NN_UCR_TRAIN_TEST'])]
#Comparison RSAST and SOTA methods(10 resampling)
#df_other_methods=df_other_methods[df_other_methods['method'].isin(['Rocket_PAPERROCKET_AVG_10RES','HCTE_PAPERROCKET_AVG_10RES','CHIEF_PAPERROCKET_AVG_10RES','DTW_R1_1NN_UCR_TRAIN_TEST'])]

df_other_methods=df_other_methods[df_other_methods['method'].isin(['Rocket_RSAST_TRAIN_TEST','RDST_RSAST_TRAIN_TEST','DTW_R1_1NN_UCR_TRAIN_TEST'])]


df_other_methods['method'].unique()

array(['DTW_R1_1NN_UCR_TRAIN_TEST', 'RDST_RSAST_TRAIN_TEST',
       'Rocket_RSAST_TRAIN_TEST'], dtype=object)

In [333]:
# Set directory where the csv files are located
#directory = os.getcwd()+'/ResultsRsast'
#directory = os.getcwd()+'/results_accuracy_per_ds'

directories=[]
directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_All')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_Extra_1')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Comparison_RSAST_Extra_2')
directories.append(os.getcwd()+'/ResultsRsast/Server17_not_used_sast')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Resampling_Comparison_RSAST')
#directories.append(os.getcwd()+'/ResultsRsast/Server16_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/ResultsRsast/Server17_Hyperparameter_Tunning')
#directories.append(os.getcwd()+'/results_accuracy_per_ds')
#directories.append(os.getcwd()+'/results_accuracy_per_ds_10000')

# Create an empty list to store the dataframes
dfs = []

# Loop through all files in the directory
for directory in directories:
    for filename in os.listdir(directory):
        # Check if the file starts with "df_overall_tunning" and ends with ".csv"
        if filename.startswith("df_all_overall_tunning") and filename.endswith(".csv"):
            # Read the csv file into a dataframe
            filepath = os.path.join(directory, filename)
            df = pd.read_csv(filepath)
            df['rpoint']=df['rpoint'].astype(str)
            df['nb_per_class']=df['nb_per_class'].astype(str)
            df['rpoint']=df['rpoint'].replace("(lenthg ts)//2","half_len")
            df['nb_per_class']=df['nb_per_class'].replace("(max instances per class)//2","half_instance")
            df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
            df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
            # Append the dataframe to the list
            dfs.append(df)

# Concatenate all the dataframes in the list into one dataframe
df_result = pd.concat(dfs, ignore_index=True)
# df_result.head(3)

/tmp/ipykernel_2911/3038894220.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_2911/3038894220.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_2911/3038894220.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_2911/3038894220.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_2911/3038894220.py:32: FutureWarning: The default value of rege

/tmp/ipykernel_2911/3038894220.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_2911/3038894220.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_2911/3038894220.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(lenthg ts\)//2","half_len")
/tmp/ipykernel_2911/3038894220.py:33: FutureWarning: The default value of regex will change from True to False in a future version.
  df['classifier_name']=df['classifier_name'].str.replace("\(max instances per class\)//2","half_instance")
/tmp/ipykernel_2911/3038894220.py:32: FutureWarning: The default value of rege

In [334]:
df_result=df_result[['accuracy','dataset_name','classifier_name','time']]

df_result=df_result[df_result["classifier_name"]==hyperparameter]

df_result.dataset_name.unique()


array(['MiddlePhalanxOutlineAgeGroup', 'FreezerSmallTrain', 'Wafer',
       'Earthquakes', 'Beef', 'DistalPhalanxOutlineAgeGroup', 'ECG200',
       'ShapeletSim', 'ItalyPowerDemand', 'ToeSegmentation2',
       'TwoLeadECG', 'ProximalPhalanxOutlineCorrect', 'CBF',
       'ToeSegmentation1', 'ShakeGestureWiimoteZ', 'TwoPatterns',
       'WordSynonyms', 'UMD', 'Strawberry', 'Mallat', 'FaceAll',
       'GunPointMaleVersusFemale', 'GunPointOldVersusYoung',
       'MelbournePedestrian', 'PLAID', 'PowerCons',
       'SonyAIBORobotSurface2', 'BirdChicken',
       'ProximalPhalanxOutlineAgeGroup', 'Herring', 'CricketY',
       'FiftyWords', 'FreezerRegularTrain', 'Trace', 'Wine',
       'MiddlePhalanxOutlineCorrect', 'BeetleFly', 'FaceFour',
       'ProximalPhalanxTW', 'SwedishLeaf', 'SonyAIBORobotSurface1',
       'PhalangesOutlinesCorrect', 'ECG5000', 'GunPoint',
       'DodgerLoopWeekend', 'Meat', 'SmoothSubspace',
       'DiatomSizeReduction', 'CricketZ', 'CricketX',
       'ChlorineConcent

In [335]:
len(df_result.dataset_name.unique())

128

In [336]:

df_other_methods=df_other_methods.rename(columns={"dataset":"dataset_name","score":"accuracy","method":"classifier_name"})
#df_other_methods = df_other_methods[df_other_methods['dataset_name'].isin(df_result.dataset_name.values)]



In [337]:
# concatenate the two dataframes
concatenated_df = pd.concat([df_other_methods, df_result])

# reset the index of the concatenated dataframe
concatenated_df = concatenated_df.reset_index(drop=True)


In [338]:
concatenated_df['classifier_name'].unique()

array(['DTW_R1_1NN_UCR_TRAIN_TEST', 'RDST_RSAST_TRAIN_TEST',
       'Rocket_RSAST_TRAIN_TEST',
       'ACF&PACF: n_random_points=10 nb_inst_per_class=10'], dtype=object)

In [339]:
concatenated_df['classifier_name'].replace(hyperparameter,"RSAST",inplace=True)
hyperparameter="RSAST"

In [340]:
concatenated_df['classifier_name'].replace("DTW_R1_1NN_UCR_TRAIN_TEST","1NN-DTW",inplace=True)
concatenated_df['classifier_name'].replace("RotF_UCR_TRAIN_TEST","RotF",inplace=True)
concatenated_df['classifier_name'].replace("ST_UCR_TRAIN_TEST","STC",inplace=True)
concatenated_df['classifier_name'].replace("SAST_PAPERSAST_TRAIN_TEST","SAST",inplace=True)
concatenated_df['classifier_name'].replace("FS_UCR_TRAIN_TEST","FS",inplace=True)
concatenated_df['classifier_name'].replace("LS_UCR_TRAIN_TEST","LS",inplace=True)
concatenated_df['classifier_name'].replace("BOSS_UCR_TRAIN_TEST","BOSS",inplace=True)
concatenated_df['classifier_name'].replace("TSBF_UCR_TRAIN_TEST","TSBF",inplace=True)
concatenated_df['classifier_name'].replace("EE_UCR_TRAIN_TEST","EE",inplace=True)
concatenated_df['classifier_name'].replace("HIVE-COTE_UCR_TRAIN_TEST","HIVE-COTE",inplace=True)
concatenated_df['classifier_name'].replace("Rocket_RSAST_TRAIN_TEST","ROCKET",inplace=True)
concatenated_df['classifier_name'].replace("TSBF_UCR_TRAIN_TEST","TSBF",inplace=True)
concatenated_df['classifier_name'].replace("ResNet_PAPERROCKET_AVG_10RES","ResNet",inplace=True)
concatenated_df['classifier_name'].replace("HCTE_PAPERROCKET_AVG_10RES","HIVE-COTE",inplace=True)
concatenated_df['classifier_name'].replace("ITime_PAPERROCKET_AVG_10RES","InceptionTime",inplace=True)
concatenated_df['classifier_name'].replace("CHIEF_PAPERROCKET_AVG_10RES","TS-CHIEF",inplace=True)
concatenated_df['classifier_name'].replace("Rocket_PAPERROCKET_AVG_10RES","ROCKET",inplace=True)
concatenated_df['classifier_name'].replace("BOSS_PAPERROCKET_AVG_10RES","BOSS",inplace=True)
concatenated_df['classifier_name'].replace("ST_PAPERROCKET_AVG_10RES","ST",inplace=True)
concatenated_df['classifier_name'].replace("R_DST_PAPERRDST_AVG_30RES","RDST",inplace=True)
concatenated_df['classifier_name'].replace("FS_UCR_RES_10","FS",inplace=True)
concatenated_df['classifier_name'].replace("LS_UCR_RES_10","LS",inplace=True)
concatenated_df['classifier_name'].replace("BOSS_UCR_RES_10","BOSS",inplace=True)
concatenated_df['classifier_name'].replace("EE_UCR_RES_10","EE",inplace=True)
concatenated_df['classifier_name'].replace("TSBF_UCR_RES_10","TSBF",inplace=True)

In [341]:
concatenated_df['classifier_name'].unique()

array(['1NN-DTW', 'RDST_RSAST_TRAIN_TEST', 'ROCKET', 'RSAST'],
      dtype=object)

In [342]:
len(concatenated_df.dataset_name.unique())

128

### Overall analysis

#### Accuracy: RSAST vs Other methods

In [343]:
max_bx=max(df_result.accuracy)
min_bx=min(df_result.accuracy)


#df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
#df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])

#print(df_to_cd.head(5))

# Plot
fig, ax = plt.subplots()


#order=list(df_to_cd)
sns.boxplot(data=df_result, y='dataset_name', x='accuracy')
#plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Axis details

ax.set(ylabel='dataset name', xlabel='accuracy', title='Accuracy Boxplot for Dataset:'+str(df_result["classifier_name"].unique()[0]))

#plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
#print(df_to_cd.describe())
#plt.tight_layout()
# save plot
plt.savefig('images_boxplot_acc/boxplot_acc_per_ds.png')

/tmp/ipykernel_2911/853197985.py:17: UserWarning: FixedFormatter should only be used together with FixedLocator
  ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')


In [344]:
filter_param=concatenated_df[concatenated_df.classifier_name==hyperparameter]

In [345]:
stats = filter_param.groupby(['dataset_name'])['accuracy'].agg(['mean', 'count', 'std'])

ci95_hi = []
ci95_lo = []
ci95_hi_lo = []

for i in stats.index:
    m, c, s = stats.loc[i]
    ci95_hi.append(m + 1.96*s/math.sqrt(c))
    ci95_lo.append(m - 1.96*s/math.sqrt(c))
    ci95_hi_lo.append(1.96*s/math.sqrt(c))

#stats['ci95_hi'] = ci95_hi
#stats['ci95_lo'] = ci95_lo
stats['ci95_interval'] = ci95_hi_lo
print(stats.round(2))

                    mean  count   std  ci95_interval
dataset_name                                        
ACSF1               0.35      5  0.02           0.02
Adiac               0.58      5  0.01           0.01
AllGestureWiimoteX  0.44      5  0.02           0.02
AllGestureWiimoteY  0.49      5  0.02           0.02
AllGestureWiimoteZ  0.38      5  0.02           0.02
...                  ...    ...   ...            ...
Wine                0.78      5  0.04           0.03
WordSynonyms        0.68      5  0.00           0.00
Worms               0.48      5  0.02           0.02
WormsTwoClass       0.63      5  0.06           0.05
Yoga                0.78      5  0.02           0.02

[128 rows x 4 columns]


In [346]:
# create a pivot table with the mean of score by hyperparameter
t_score_by_method=pd.pivot_table(concatenated_df, values='accuracy', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')
#t_score_by_method=score_by_method.transpose()[[hyperparameter,"SAST"]].round(2)
#t_score_by_method=score_by_method.transpose().round(2)
t_score_by_method.iloc[0:25,:]


classifier_name,1NN-DTW,RDST_RSAST_TRAIN_TEST,ROCKET,RSAST
dataset_name,,,,
ACSF1,NaN,0.918000,0.890000,0.350000
Adiac,0.603581,0.727877,0.784143,0.576982
AllGestureWiimoteX,NaN,0.768857,0.779429,0.444571
AllGestureWiimoteY,NaN,0.758857,0.768857,0.494857
AllGestureWiimoteZ,NaN,0.710286,0.758000,0.375143
ArrowHead,0.702857,0.859429,0.806857,0.788571
BME,NaN,0.993333,1.000000,0.832000
Beef,0.633333,0.833333,0.820000,0.680000
BeetleFly,0.700000,0.950000,0.900000,0.770000


In [347]:
t_score_by_method.iloc[25:49,:]

classifier_name,1NN-DTW,RDST_RSAST_TRAIN_TEST,ROCKET,RSAST
dataset_name,,,,
DodgerLoopDay,NaN,0.635000,0.612500,0.567500
DodgerLoopGame,NaN,0.862319,0.846377,0.818841
DodgerLoopWeekend,NaN,0.978261,0.975362,0.953623
ECG200,0.770000,0.898000,0.920000,0.880000
ECG5000,0.924444,0.945511,0.947289,0.937289
ECGFiveDays,0.767712,0.997909,1.000000,0.995354
EOGHorizontalSignal,NaN,0.675138,0.640884,0.369613
EOGVerticalSignal,NaN,0.545304,0.543646,0.333149
Earthquakes,0.719424,0.722302,0.755396,0.717986


In [348]:
t_score_by_method.iloc[49:75,:]

classifier_name,1NN-DTW,RDST_RSAST_TRAIN_TEST,ROCKET,RSAST
dataset_name,,,,
GesturePebbleZ1,NaN,0.903488,0.908140,0.870930
GesturePebbleZ2,NaN,0.888608,0.835443,0.854430
GunPoint,0.906667,1.000000,1.000000,0.980000
GunPointAgeSpan,NaN,0.996835,0.996835,0.969620
GunPointMaleVersusFemale,NaN,1.000000,0.997468,0.987975
GunPointOldVersusYoung,NaN,1.000000,0.988571,0.961270
Ham,0.466667,0.712381,0.714286,0.742857
HandOutlines,0.881081,0.950270,0.945946,0.910270
Haptics,0.376623,0.576623,0.525325,0.385065


In [349]:
t_score_by_method.iloc[76:,:]

classifier_name,1NN-DTW,RDST_RSAST_TRAIN_TEST,ROCKET,RSAST
dataset_name,,,,
MixedShapesSmallTrain,NaN,0.967588,0.937814,0.807093
MoteStrain,0.834665,0.926358,0.916933,0.884185
NonInvasiveFetalECGThorax1,0.790331,0.927532,0.955725,0.833384
NonInvasiveFetalECGThorax2,0.864631,0.939746,0.968448,0.915013
OSULeaf,0.590909,0.980165,0.935537,0.559504
OliveOil,0.833333,0.886667,0.926667,0.800000
PLAID,NaN,0.956052,0.892365,0.779888
PhalangesOutlinesCorrect,0.728438,0.807459,0.833566,0.789977
Phoneme,0.228376,0.339241,0.277637,0.187447


In [350]:
t_score_by_method.describe().round(2)

classifier_name,1NN-DTW,RDST_RSAST_TRAIN_TEST,ROCKET,RSAST
count,85.00,128.00,128.00,128.00
mean,0.74,0.86,0.85,0.75
std,0.17,0.14,0.15,0.21
min,0.23,0.34,0.28,0.11
25%,0.64,0.77,0.78,0.64
50%,0.75,0.90,0.90,0.79
75%,0.86,0.98,0.97,0.90
max,1.00,1.00,1.00,1.00


In [351]:
t_score_by_method=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=t_score_by_method.transpose()#[[hyperparameter,"SAST"]].round(2)

In [352]:
#wins of RSAST
#t_score_by_method[0<(t_score_by_method[hyperparameter].round(2)-t_score_by_method["SAST"].round(2))].index

In [353]:
#concatenated_df['classifier_name']=concatenated_df['classifier_name'].str.split('_').str[0]

In [354]:
pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name'], aggfunc='var').round(4)

,accuracy
classifier_name,
1NN-DTW,0.0284
RDST_RSAST_TRAIN_TEST,0.0203
ROCKET,0.0227
RSAST,0.0432


In [355]:
pv_all=pd.pivot_table(concatenated_df,index='dataset_name',values='accuracy',aggfunc="count")
ds_complete=pv_all[pv_all.accuracy==max(pv_all.accuracy)].reset_index().dataset_name.unique()
ds_complete
concatenated_df=concatenated_df[concatenated_df.dataset_name.isin(ds_complete)]

In [356]:
#concatenated_df=concatenated_df[concatenated_df.dataset_name!="Fungi"]
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
pd.pivot_table(df_perf, values='accuracy', index=['classifier_name'], aggfunc='count')

,accuracy
classifier_name,
1NN-DTW,85
RDST_RSAST_TRAIN_TEST,85
ROCKET,85
RSAST,85


In [357]:
#concatenated_df['classifier_name']=concatenated_df['classifier_name'].str.split('_').str[0]
score_by_method=pd.pivot_table(df_perf, values='accuracy', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')
score_by_method

classifier_name,1NN-DTW,RDST_RSAST_TRAIN_TEST,ROCKET,RSAST
dataset_name,,,,
Adiac,0.603581,0.727877,0.784143,0.576982
ArrowHead,0.702857,0.859429,0.806857,0.788571
Beef,0.633333,0.833333,0.820000,0.680000
BeetleFly,0.700000,0.950000,0.900000,0.770000
BirdChicken,0.750000,0.900000,0.900000,0.850000
...,...,...,...,...
Wine,0.574074,0.918519,0.818519,0.777778
WordSynonyms,0.648903,0.746395,0.751097,0.684953
Worms,0.584416,0.688312,0.729870,0.480519


In [358]:
#hyperparameter='RDST_30RES'

In [359]:
t_score_by_method=pd.pivot_table(df_perf, values='accuracy', index=['classifier_name'],columns=['dataset_name'], aggfunc='mean')
t_score_by_method=t_score_by_method.transpose()#[[hyperparameter,"SAST"]].round(2)
for methods in t_score_by_method.columns.drop(hyperparameter):
    print("method: "+methods+" vs "+hyperparameter)
    total_wins=len(t_score_by_method[0<(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("winning "+hyperparameter+": "+str(total_wins))
    total_loss=len(t_score_by_method[0>(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("losse "+hyperparameter+": "+str(total_loss))
    total_draws=len(t_score_by_method[0==(t_score_by_method[hyperparameter].round(2)-t_score_by_method[methods].round(2))])
    print("draws: "+str(total_draws))

method: 1NN-DTW vs RSAST
winning RSAST: 56
losse RSAST: 22
draws: 7
method: RDST_RSAST_TRAIN_TEST vs RSAST
winning RSAST: 5
losse RSAST: 70
draws: 10
method: ROCKET vs RSAST
winning RSAST: 4
losse RSAST: 74
draws: 7


In [360]:
for method in score_by_method.columns.drop(hyperparameter):
    y=score_by_method[hyperparameter]
    x=score_by_method[method]

    # set the style of the plot
    plt.style.use('seaborn')

    # create the plot
    fig, ax = plt.subplots(figsize=(8, 6))

    # plot the line
    ax.plot([0,1], [0,1])

    # plot the points
    ax.scatter(x, y, color='blue', edgecolor='blue', s=30, zorder=10)

    # add labels and title
    ax.set_ylabel(hyperparameter)
    ax.set_xlabel(method)
    ax.set_title('Accuracy')


    # set the background color
    ax.set_facecolor('lightgrey')
    plt.savefig('images_one_vs_one_comparison/'+hyperparameter+'vs'+method+'.png')


/tmp/ipykernel_2911/2505631233.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')
/tmp/ipykernel_2911/2505631233.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


/tmp/ipykernel_2911/2505631233.py:6: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [361]:
# create a pivot table with the mean of time by hyperparameter
time_per_method=pd.pivot_table(df_result, values='time', columns=['classifier_name'],index=['dataset_name'], aggfunc='mean')/60
time_per_method

classifier_name,ACF&PACF: n_random_points=10 nb_inst_per_class=10
dataset_name,
ACSF1,0.030428
Adiac,37.744124
AllGestureWiimoteX,0.009497
AllGestureWiimoteY,0.009813
AllGestureWiimoteZ,0.009733
...,...
Wine,0.262070
WordSynonyms,36.278904
Worms,0.009722


#### Generate CD Diagram comparison

In [362]:
concatenated_df.classifier_name.unique()

array(['1NN-DTW', 'RDST_RSAST_TRAIN_TEST', 'ROCKET', 'RSAST'],
      dtype=object)

In [363]:

from cd_function import *
df_perf=pd.pivot_table(concatenated_df, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
df_perf=df_perf.reset_index()
draw_cd_diagram(df_perf, labels=True, fname='images_cd_diagram/cd-diagram_overall_comparison_other_method.png')


{0: '1NN-DTW', 1: 'RSAST', 2: 'RDST_RSAST_TRAIN_TEST', 3: 'ROCKET'}
[2, 3]


In [364]:
df_perf["classifier_name"].unique()

array(['1NN-DTW', 'RDST_RSAST_TRAIN_TEST', 'ROCKET', 'RSAST'],
      dtype=object)

In [365]:
_, average_ranks, _ = wilcoxon_holm(df_perf=df_perf)
min_rank= min(average_ranks)
average_ranks=pd.DataFrame(average_ranks)
average_ranks

,0
1NN-DTW,3.541176
RSAST,3.158824
RDST_RSAST_TRAIN_TEST,1.705882
ROCKET,1.594118


In [366]:
uca_ds=pd.read_excel("DataSetsUCLASummary.xlsx")

uca_ds.columns

Index(['USED SAST', 'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'N RUNS S17_SAST_DS_all', 'N RUNS S17_SAST_DS_extra_1',
       'N RUNS S17_SAST_DS_extra_2', 'N RUNS S17_RDST_ROCKET',
       'N RUNS S17_RSAST_R10', 'ID', 'Name', 'Type', 'Train ', 'Test ',
       'Class', 'Length', 'RSAST (10,10) WORST SAST?', 'O(|c|nm²)',
       'O(|c|nm³)', 'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 29'],
      dtype='object')

In [367]:
concatenated_df=concatenated_df.merge(uca_ds, left_on=['dataset_name'],right_on=['Name'])
concatenated_df.columns

Index(['dataset_name', 'accuracy', 'classifier_name', 'time', 'USED SAST',
       'BAKE OFF', 'N RUNS S17_S16_HT', 'N RUNS S17_SAST_DS',
       'N RUNS S17_SAST_DS_all', 'N RUNS S17_SAST_DS_extra_1',
       'N RUNS S17_SAST_DS_extra_2', 'N RUNS S17_RDST_ROCKET',
       'N RUNS S17_RSAST_R10', 'ID', 'Name', 'Type', 'Train ', 'Test ',
       'Class', 'Length', 'RSAST (10,10) WORST SAST?', 'O(|c|nm²)',
       'O(|c|nm³)', 'Approx Time (hours)', 'L25', 'L50', 'L75', 'ED (w=0)',
       'DTW (learned_w) ', 'DTW (w=100)', 'Default rate', 'Data donor/editor',
       '50/L', 'Unnamed: 29'],
      dtype='object')

In [368]:
acc_per_type=pd.pivot_table(concatenated_df, values="accuracy", index=["dataset_name","Type",'classifier_name'])
#acc_per_type=concatenated_df
acc_per_type=acc_per_type.reset_index()

In [369]:
max_bx=max(acc_per_type.accuracy)
min_bx=min(acc_per_type.accuracy)


#df_to_cd=pd.pivot_table(df_to_cd, index=['dataset_name','classifier_name'], columns=['rpoint'],values='accuracy')
#df_to_cd = df_to_cd.reindex(columns=["1","10","30","50","100","half_len"])

#print(df_to_cd.head(5))

# Plot
fig, ax = plt.subplots()


#order=list(df_to_cd)
sns.boxplot(data=acc_per_type, y='Type', x='accuracy', hue ='classifier_name')
#plt.boxplot(df_to_cd, labels=list(df_to_cd), showfliers=False)
#ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Axis details

ax.set(ylabel='dataset name', xlabel='accuracy', title='Accuracy Boxplot for Dataset:'+str(df_result["classifier_name"].unique()[0]))

#plt.xticks([1, 2, 3, 4, 5, 6],list(df_to_cd) )
#print(df_to_cd.describe())
plt.tight_layout()
# save plot
plt.savefig('images_boxplot_acc_per_type/boxplot_acc_per_type.png')

In [370]:
pd.pivot_table(acc_per_type,index='Type', aggfunc=lambda x: len(x.unique()))

,accuracy,classifier_name,dataset_name
Type,,,
Device,24,4,6
ECG,24,4,6
Image,113,4,29
Motion,54,4,14
Sensor,64,4,18
Simulated,16,4,5
Spectro,24,4,7
